# Mineração de Regras de Associação

### Importação das Bibliotecas:

In [68]:
import pandas as pd
import numpy as np
from apyori import apriori

### Importação dos Datasets:

In [ ]:
df = pd.read_csv('../../data/dataset_new_with_outliers.csv')
df

--------------------------------------------

### Formatação dos Dados:

* Definindo Regras de Formatação:

% == 0 = 0, % > 0 = 1;

CI < 3 = 0, CI >= 3 = 1;

IGC < 3 = 0, IGC >= 3 = 1;

variacao mat <= 0 = 0, variacao mat > 0 = 1;

% FIES == 0 = 0, % FIES > 0 = 1;

CURSOS == 0 = 0, CURSOS > 0 = 1;

saldo <= 0 = 0, saldo > 0 = 1;

* Formatando dataset:

In [ ]:
df_formated = df[['Natureza', 'Situacao', '%', 'CI N', 'CI V', 'IGC_N', 'IGC_M', 'IGC_V', 'EAD_17', 'variacao mat', '% FIES', 'CURSOS', 'saldo 2017']]

df_formated["%"] = np.where(df_formated["%"] == 0, '5AnosSemAR-Não', '5AnosSemAR-Sim')

df_formated["CI N"] = np.where(df_formated["CI N"] < 3, 'CI_N-Insuficiente', 'CI_N-Suficiente')
df_formated["CI V"] = np.where(df_formated["CI V"] < 3, 'CI_V-Insuficiente', 'CI_V-Suficiente')

df_formated["IGC_N"] = np.where(df_formated["IGC_N"] < 3, 'IGC_N-Insuficiente', 'IGC_N-Suficiente')
df_formated["IGC_M"] = np.where(df_formated["IGC_M"] < 3, 'IGC_M-Insuficiente', 'IGC_M-Suficiente')
df_formated["IGC_V"] = np.where(df_formated["IGC_V"] < 3, 'IGC_V-Insuficiente', 'IGC_V-Suficiente')

df_formated["variacao mat"] = np.where(df_formated["variacao mat"] <= 0, 'Matriculas-Diminuiu', 'Matriculas-Aumentou')
df_formated["% FIES"] = np.where(df_formated["% FIES"] == 0, 'Matriculas_FIES-Não', 'Matriculas_FIES-Sim')
df_formated["CURSOS"] = np.where(df_formated["CURSOS"] == 0, 'Ofereceu_Cursos-Não', 'Ofereceu_Cursos-Sim')
df_formated["saldo 2017"] = np.where(df_formated["saldo 2017"] <= 0, 'Saldo_2017-Não', 'Saldo_2017-Sim')

df_formated["Natureza"] = np.where(df_formated["Natureza"] == 0, 'Fins_Lucrativos-Não', 'Fins_Lucrativos-Sim')
df_formated["Situacao"] = np.where(df_formated["Situacao"] == 0, 'Situação-Extinta', 'Situação-Ativa')
df_formated["EAD_17"] = np.where(df_formated["EAD_17"] == 0, 'Ofereceu_EAD_17-Não', 'Ofereceu_EAD_17-Sim')

df_formated = df_formated.drop(columns={'CI N', 'CI V', 'CURSOS'})
df_formated

* Transformando em transações:

In [71]:
transactions = []
for i in range(len(df_formated)):
    transactions.append([df_formated.values[i,j] for j in range(df_formated.shape[1])])

transactions

[['Fins_Lucrativos-Sim',
  'Situação-Ativa',
  '5AnosSemAR-Sim',
  'IGC_N-Insuficiente',
  'IGC_M-Insuficiente',
  'IGC_V-Insuficiente',
  'Ofereceu_EAD_17-Não',
  'Matriculas-Diminuiu',
  'Matriculas_FIES-Não',
  'Saldo_2017-Não'],
 ['Fins_Lucrativos-Sim',
  'Situação-Ativa',
  '5AnosSemAR-Sim',
  'IGC_N-Insuficiente',
  'IGC_M-Insuficiente',
  'IGC_V-Insuficiente',
  'Ofereceu_EAD_17-Sim',
  'Matriculas-Diminuiu',
  'Matriculas_FIES-Não',
  'Saldo_2017-Sim'],
 ['Fins_Lucrativos-Não',
  'Situação-Ativa',
  '5AnosSemAR-Sim',
  'IGC_N-Insuficiente',
  'IGC_M-Insuficiente',
  'IGC_V-Insuficiente',
  'Ofereceu_EAD_17-Não',
  'Matriculas-Diminuiu',
  'Matriculas_FIES-Não',
  'Saldo_2017-Não'],
 ['Fins_Lucrativos-Não',
  'Situação-Ativa',
  '5AnosSemAR-Sim',
  'IGC_N-Insuficiente',
  'IGC_M-Insuficiente',
  'IGC_V-Insuficiente',
  'Ofereceu_EAD_17-Sim',
  'Matriculas-Diminuiu',
  'Matriculas_FIES-Não',
  'Saldo_2017-Sim'],
 ['Fins_Lucrativos-Sim',
  'Situação-Ativa',
  '5AnosSemAR-Sim',
  '

-----------------------------------------

### Algoritmo Apriori:

In [72]:
rules = apriori(transactions, min_support=0.001, min_confidence=0.8, min_lift=2)
results = list(rules)
len(results)

6550

In [73]:
A = []
B = []
support = []
confidence = []
lift = []

for result in results:
    s = result[1]
    result_rules = result[2]
    for result_rule in result_rules:
        a = list(result_rule[0])
        b = list(result_rule[1])
        c = result_rule[2]
        l = result_rule[3]

        A.append(a)
        B.append(b)
        support.append(s)
        confidence.append(c)
        lift.append(l)

In [ ]:
df_rules = pd.DataFrame({'Rule': A, 'Implication': B, 'Support': support, 'Confidence': confidence, 'Lift': lift})
df_rules['Rule'] = df_rules['Rule'].astype('string')
df_rules['Implication'] = df_rules['Implication'].astype('string')
df_rules.to_csv('../../data/rules_association_teste.csv', index=False)
df_rules

------------------------------

### Avaliando Regras de Associação:

In [ ]:
df_rules = pd.read_csv('../../data/rules_association_teste.csv')
pd.set_option('display.max_rows', None)
df_rules.head(None)
df_rules

In [77]:
df_rules.sort_values(by='Confidence', ascending=False).loc[df_rules['Implication'].str.contains('Situação-Ativa')==True].to_csv('../../data/association_rules_Ativa.csv', index=False)
df_rules.sort_values(by='Confidence', ascending=False).loc[df_rules['Implication'].str.contains('Situação-Extinta')==True].to_csv('../../data/association_rules_Extinta.csv', index=False)